In [1]:
# importing libraries

import requests
import pyodbc
from datetime import datetime, timedelta

In [2]:
# 1. defining cities and coordinates in a dictionery called 'CITIES'

CITIES = {
    "London": {"lat": 51.5074, "lon": -0.1278},
    "New York": {"lat": 40.7128, "lon": -74.0060},
    "Sydney": {"lat": -33.8688, "lon": 151.2093},
    "Lahore": {"lat": 31.5204, "lon": 74.3587},
    "Dubai": {"lat": 25.276987, "lon": 55.296249}
}


In [3]:
# 2. establishing a connection with SQL server using a try-except block method

try:
    sql_connection = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=localhost;'
        'DATABASE=WeatherDataWarehouse;'
        'Trusted_Connection=yes;'
    )
    print("✅ SQL Connection Successful")
except Exception as e:
    print(f"❌ Connection Failed: {str(e)}")
    raise

✅ SQL Connection Successful


In [ ]:
# 3. Process each city
for city, coords in CITIES.items():
    # 4. Get last recorded date
    cursor = sql_connection.cursor()
    cursor.execute("SELECT MAX(date) FROM fact_weather WHERE city_id = (SELECT city_id FROM dim_city WHERE city_name = ?)", city)
    last_date = cursor.fetchone()[0] or datetime(2000, 1, 1).date()
    
    # 5. Calculate date range
    start_date = last_date + timedelta(days=1)
    end_date = datetime.now().date()
    if start_date > end_date:
        continue
    
    # 6. Fetch weather data from API
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={coords['lat']}&longitude={coords['lon']}&start_date={start_date}&end_date={end_date}&daily=temperature_2m_max,temperature_2m_min,precipitation_sum"
    
    response = requests.get(url)
    data = response.json()

    '''
    - Checking if the response status code is 200.
   - Checking if the JSON has a key 'error' that is True.
    '''
    if response.status_code == 200 and 'daily' in data:
        daily_data = data['daily']
        print(f"Data fetched for {city}")
    else:
        print(f"ERROR for {city}: {data.get('reason', 'Unknown error')}")  # Key insight

   

Data fetched for London
Data fetched for New York
Data fetched for Sydney
Data fetched for Lahore
ERROR for Dubai: Minutely API request limit exceeded. Please try again in one minute.


In [ ]:
 # 7. Insert into staging table
    for i in range(len(daily_data['time'])):
        cursor.execute("""
            INSERT INTO stg_weather_raw (city_name, date, temp_max, temp_min, precipitation)
            VALUES (?, ?, ?, ?, ?)
        """, city, 
           daily_data['time'][i],
           daily_data['temperature_2m_max'][i],
           daily_data['temperature_2m_min'][i],
           daily_data['precipitation_sum'][i])
    
    sql_connection.commit()

# 8. Clean up
#sql_connection.close()